#  Chameleon Facility Ports: Layer 2

A primary purpose of FABRIC is to be a platform for experimentation with innovative ideas about the future of the Internet.  One ability that makes FABRIC standout from other testbeds is its facility ports that connect FABRIC experiments to external facilities using low-level high bandwidth circuits.


<img src="./figs/stitching_triangle.png" width="60%"><br>

Experimental topologies, like the one above, can connect multiple computational testbeds to a large instrument, or even your institution, with dedicated high-bandwidth network links and smart in-network processing.

This notebook introduces you to stitching experiments spanning [Chameleon](https://www.chameleoncloud.org/) and FABRIC. What you will learn today is not a full-fledged experiment but rather a building block that you can apply to your own experiments.

Using this notebook, you will create a network on Chameleon at TACC that is connected to FABRIC's FABnetv4 Layer 3 network service. Once deployed, you will be able to create nodes on Chameleon and FABRIC that communicate directly over FABRIC's dedicated links.


The goal for today is to create an experiment that resembles the following figure.  There will be a small set of nodes on Chameleon at TACC and a couple sets of nodes at other FABRIC sites.


<img src="./figs/fabnet_stitch.png" width="60%"><br>




## Chameleon Environment


One of the main challenges to creating slices that span multiple testbeds is the need to learn how to use multiple testbed APIs.  Today you will need to learn about the Chameleon python API.  Conveniently, FABRIC and Chameleon both use a Python API and can, simultaneously, use a single Jupyter notebook.

This section describes how to configure Chameleon inside the FABRIC JupyterHub.

We assume you have already configured your environment to use FABRIC.  If this is your first time using FABRIC, you may need to follow the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook to complete the configuration.


### Create a Chameleon CLI Password

Chameleon has a good set of directions for creating a Chameleon CLI password. Follow the Chameleon [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#cli-authentication) to create a Chameleon CLI password. 

### Create an Openrc File

After creating your CLI Password, download a your `Chameleon-openrc.sh` file by following these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#the-openstack-rc-script). 

Make sure to get the openrc file that targets the TACC Chameleon site.

### Configure your FABRIC Environment with your Chameleon Information

Set the following environment vars to the values as found in your custom Chameleon-openrc.sh file.  The password should be set to your new CLI password.




In [ ]:
import os

os.environ["OS_USERNAME"]='<username>'
os.environ["OS_PASSWORD"]='<password>'

os.environ["OS_AUTH_URL"]='https://chi.uc.chameleoncloud.org:5000/v3'
os.environ["OS_IDENTITY_API_VERSION"]='3'
os.environ["OS_INTERFACE"]='public'
os.environ["OS_PROTOCOL"]="openid"
os.environ["OS_AUTH_TYPE"]="v3oidcpassword"
os.environ["OS_IDENTITY_PROVIDER"]="chameleon"
os.environ["OS_DISCOVERY_ENDPOINT"]="https://auth.chameleoncloud.org/auth/realms/chameleon/.well-known/openid-configuration"
os.environ["OS_CLIENT_ID"]="keystone-uc-prod"
os.environ["OS_ACCESS_TOKEN_TYPE"]="access_token"
os.environ["OS_CLIENT_SECRET"]="none"
os.environ["OS_REGION_NAME"]="CHI@UC"


Alternatively, edit the Chameleon openrc file in this notebook's folder and add your information. Then you can read the environment var from the file with `load_dotenv`.   Loading the variable is not necessary but allows you to keep your CLI password from appearing in your notebook.

<div class="alert alert-block alert-info">
<b>Tip:</b> Put your Chameleon openrc file in your fabric_config folder with your FABRIC keys.  Then reference it here with a full path: f'{os.environ["HOME"]}/work/fabric_config/chameleon-openrc-tacc.sh'

</div>

In [ ]:
from dotenv import load_dotenv
import os

# Load the environment variables from the myenvfile.txt file
load_dotenv(f'chameleon-openrc-tacc.sh');


## Import the Chameleon and FABRIC Libraries

The libraries are already installed in the FABRIC Jupyter Hub. Directions on the Chameleon website describe how to install them on your local machine.




### Import Libraries

In [ ]:
#General imports
import os
import json
import traceback
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
from datetime import datetime, timedelta
from dateutil import tz
import time

# Chameleon Library
import chi
import chi.lease 
from chi.server import *
from chi.lease import *
from chi.network import *

# FABRIC Library
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

### Chameleon Experiment Variable

Specify some name variables to use throughout the notebook.

Make sure to set your Chameleon ssh key and an active reservation for Chameleon servers.

Put your username at the beginning of each name so that we can identify which networks and nodes belong to each user.


In [ ]:
# Chameleon Config
chameleon_prefix =  f'{os.environ["OS_USERNAME"]}_fabric_stitch_l2'
chameleon_server_name = chameleon_prefix+'_server'
chameleon_network_name = chameleon_prefix+'_net'
chameleon_subnet_name = chameleon_prefix+'_subnet'
chameleon_router_name = chameleon_prefix+'_router'
chameleon_lease_name = chameleon_prefix+'_lease'

chameleon_image_name='CC-Ubuntu20.04'
chameleon_server_count=1

chameleon_key_name='<Your_Chameleon_Key>'
chameleon_server_reservation_id = '<Your_Chameleon_Server_Reservation>'

### FABRIC Experiment Variables

We will use the TACC Chameleon Facility Port.  The Chicago Chameleon Facility Port name is listed for future reference.

In [ ]:
# Create a FABlib manager
fablib = fablib_manager()
fablib.show_config()

# TACC
fabric_site='TACC'
fabric_slice_name="chameleon_tacc_stitch_l2"
faciliy_port='Chameleon-TACC'

# OR Chicago
#fabric_site='STAR'
#fabric_slice_name="chameleon_chicagostitch_l2"
#faciliy_port='Chameleon-StarLight'


## Pick you own L3 Subnet

Reserve a Chameleon network at TACC and specify the `fabric` stitch provider. 

In [ ]:
#Network Config
subnet = IPv4Network("192.168.100.0/24")

fabric_allocation_pool_start=IPv4Address('192.168.100.200')
fabric_allocation_pool_end=IPv4Address('192.168.100.250')
fabric_available_ips=[]
for ip_int in range(int(fabric_allocation_pool_start),int(fabric_allocation_pool_end)+1):
    fabric_available_ips.append(IPv4Address(ip_int))
    
chameleon_allocation_pool_start='192.168.100.100'
chameleon_allocation_pool_end='192.168.100.150'
chameleon_gateway_ip='192.168.100.1'

## Create the Chameleon Network

Reserve a Chameleon network at TACC and specify the `fabric` stitch provider. 

In [ ]:
BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
reservation_list.append(
        {
            "resource_type": "network",
            "network_name": chameleon_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", 'fabric']
            ),
        }
)

# Create the lease
chameleon_lease = chi.lease.create_lease(chameleon_lease_name,
                                  reservations=reservation_list,
                                  start_date=start_date,
                                  end_date=end_date)
    
#Print the lease info
chameleon_network_reservation_id = [reservation for reservation in chameleon_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
print(f"chameleon_network_reservation_id: {chameleon_network_reservation_id}")


#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The `get_network`
call will fail if a network with that name does not yet exist. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

After the network becomes active, you will have a raw L2 VLAN network that connects to Chameleon to FABRIC.

Note the VLAN that you were assigned by Chameleon.  You will need this in the next step to create the FABRIC half of the stitch.




In [ ]:
network_vlan = None
while network_vlan == None:
    try:
        #Get the network
        chameleon_network = chi.network.get_network(chameleon_network_name)

        #Get the network ID
        chameleon_network_id = chameleon_network['id']
        print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        network_vlan = chameleon_network['provider:segmentation_id']
        print(f'network_vlan: {network_vlan}')
    except:
        print(f'Chameleon Network is not ready. Trying again!')
        time.sleep(10)           

#### Add a subnet to the reserved network

In [ ]:
chameleon_subnet = chi.network.create_subnet(chameleon_subnet_name, chameleon_network_id, 
                                             cidr=str(subnet),
                                             allocation_pool_start=chameleon_allocation_pool_start,
                                             allocation_pool_end=chameleon_allocation_pool_end,
                                             gateway_ip=chameleon_gateway_ip)

print(f"subnet name  : {chameleon_subnet['name']}")
print(f"subnet       : {chameleon_subnet['cidr']}")
print(f"gateway_ip   : {chameleon_subnet['gateway_ip']}")
#print(json.dumps(chameleon_subnet, indent=2))

#### Add a Router and add it to the Subnet

In [ ]:
chameleon_router = chi.network.create_router(chameleon_router_name, gw_network_name='public')

#print(json.dumps(chameleon_router, indent=2))
print(f"router name  : {chameleon_router['name']}")

connection_port = chi.network.add_subnet_to_router_by_name(chameleon_router_name, chameleon_subnet_name)

print(f"connection_port id  : {connection_port['port_id']}")

## Create FABRIC Node and Link

Create the FABRIC slice. This is a small slice that does not even contain a node.  It only contain the facility port and a FABnetv4 network on the closest FABRIC site.


## Create FABRIC Node and Link

In [ ]:
#Create a slice
fabric_slice = fablib.new_slice(name=fabric_slice_name)

# Create a FABRIC network
net1 = fabric_slice.add_l2network(name='net1', subnet=subnet)

# Add a Node
fabric_node = fabric_slice.add_node(name='Node1', site=fabric_site)
fabric_node_iface = fabric_node.add_component(model='NIC_Basic', name=f"nic1").get_interfaces()[0]
fabric_node_iface.set_mode('config')
net1.add_interface(fabric_node_iface)
fabric_node_iface.set_ip_addr(fabric_available_ips.pop(0))
    
# Add the Facility Port
fabric_facility_port = fabric_slice.add_facility_port(name=faciliy_port, site=fabric_site, vlan=str(network_vlan))
fabric_facility_port_iface = fabric_facility_port.get_interfaces()[0]
fabric_facility_port_iface.set_mode('manual')
net1.add_interface(fabric_facility_port_iface)

#Submit the Request
fabric_slice.submit()

## (Optional) Test the Link

We can test the facility port link by pinging the Chameleon gateway IP from the FABRIC VM.

In [ ]:
try:
    fabric_node = fabric_slice.get_node(name='Node1')     
    fabric_node_iface = fabric_node.get_interface(network_name=f'net1') 


    stdout, stderr = fabric_node.execute(f'ping -c 5 {chameleon_gateway_ip}')
    
except Exception as e:
    print(f"Exception: {e}")

## Start Chameleon Servers

In [ ]:
#import chi.server

servers = []

for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i+1}"
    # Create the server
    servers.append(chi.server.create_server(server_name, 
                                  reservation_id=chameleon_server_reservation_id, 
                                  network_name=chameleon_network_name, 
                                  image_name=chameleon_image_name,
                                  key_name=chameleon_key_name
                                 ))
    
# Wait until the server is active
for server in servers:
    print(f'Waiting for server: {server.name}')
    chi.server.wait_for_active(server.id)
print('Done!')

## Get the Chameleon Server Fixed IPs

In [ ]:
#get fixed ips
fixed_ips={}
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i+1}"
    server_id = get_server_id(server_name)
    fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    fixed_ips[server_name]=fixed_ip

for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')

## Test the Link by Pinging the Chameleon Nodes

In [ ]:
fabric_node = fabric_slice.get_node(name='Node1')     

for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')
    
    stdout, stderr = fabric_node.execute(f'ping -c 5 {fixed_ip}')
   

# Clean Up 

## Delete Chameleon Resources

Delete the servers

In [ ]:
for i in range(chameleon_server_count):
    server_name=f"{chameleon_server_name}_{i+1}"
    chi.server.delete_server(get_server_id(server_name))

#### De-configure Network

In [ ]:
router_id = chameleon_router['id']
subnet_id = chameleon_subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass

## Release Lease

In [ ]:
chi.lease.delete_lease(chameleon_lease['id'])

## Delete FABRIC Slice

In [ ]:
try:
    fabric_slice.delete()
except Exception as e:
    print(f"Exception: {e}")